In [1]:
import sys
sys.path.append('..')
from osp import *

In [4]:
df=get_pmla_df()

100%|██████████| 71902/71902 [00:02<00:00, 29445.68it/s]


In [5]:
df.decade.value_counts()

decade
2000    3104
2010    2813
1970    2556
1990    2541
1980    2513
1950    2172
1960    2106
1930    2105
1940    1956
1920    1605
1910    1233
1900     639
Name: count, dtype: int64

In [6]:
def iter_pmla_sample():
    df = get_pmla_df().sample(frac=1)
    decs = list(df.groupby('decade'))
    maxn = max(len(decdf) for dec,decdf in decs)
    for i in range(maxn):
        for dec,decdf in decs:
            if len(decdf) > i:
                yield decdf.iloc[i]

In [7]:
row = next(iter_pmla_sample())

100%|██████████| 71902/71902 [00:02<00:00, 27461.61it/s]


In [8]:
def parse_pmla_row(row, force=False):
    id = row['id']
    if not force and id in PMLA_STASH:
        return stanza.Document.from_serialized(PMLA_STASH[id])
    
    text = row['fullText']
    article = filter_pmla_pages(text)
    doc = get_nlp_doc(article)
    PMLA_STASH[id] = doc.to_serialized()
    return doc

In [10]:
doc = parse_pmla_row(row)
tree_stats = pd.DataFrame(get_tree_stats(sent) for sent in doc.sentences)
tree_stats

,num_words,height,num_independent_clauses,num_dependent_clauses,num_words_in_dependent_clauses,num_words_in_independent_clauses,num_punct,num_punct_colon,num_punct_comma,num_parens,num_words_adjective,num_words_noun,num_words_adverb,num_words_verb
0,9,5,0,0,0,9,3,0,1,0,1.0,4.0,NaN,NaN
1,9,6,0,0,0,9,1,0,0,0,NaN,4.0,NaN,NaN
2,35,14,1,1,10,25,2,1,1,0,3.0,11.0,1.0,4.0
3,39,22,1,2,19,20,4,0,3,0,1.0,10.0,2.0,4.0
4,20,8,1,0,0,20,4,0,3,0,1.0,3.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,33,11,3,0,0,33,7,0,6,0,1.0,11.0,NaN,3.0
323,1,3,0,0,0,1,0,0,0,0,NaN,NaN,NaN,NaN
324,30,14,1,2,34,-4,3,0,2,0,1.0,9.0,1.0,4.0
325,16,15,2,0,0,16,1,0,0,0,NaN,4.0,1.0,4.0


In [13]:
tree_stats.mean()

num_words                           31.281346
height                              11.975535
num_independent_clauses              1.458716
num_dependent_clauses                0.706422
num_words_in_dependent_clauses      11.740061
num_words_in_independent_clauses    19.541284
num_punct                            4.859327
num_punct_colon                      0.223242
num_punct_comma                      2.984709
num_parens                           0.657492
num_words_adjective                  1.727273
num_words_noun                       8.907285
num_words_adverb                     1.898089
num_words_verb                       3.329670
dtype: float64

In [14]:
done = set(PMLA_STASH.keys_l())
len(done)

931

In [15]:
sample = [row for row in iter_pmla_sample() if row['id'] not in done]
len(sample)

100%|██████████| 71902/71902 [00:03<00:00, 21723.62it/s]


24412

In [ ]:
for row in tqdm(sample):
    parse_pmla_row(row)

  7%|▋         | 1730/24412 [14:29:08<331:15:59, 52.58s/it]